# Sentiment Model
Weight and Notebook at https://github.com/Ayushk4/JuliaCon20_Talk

In [ ]:
using Flux, BSON, WordTokenizers

In [ ]:
mutable struct SentimentModel{WE, L1, L2, D}
    vocab_dict::Dict{String,Integer} # Dict that maps words to indices in W_word_Embed
    W_Embed::WE # Fine Tuned GloVe Embeddings
    lstm1::L1 # Forward LSTM 1
    lstm2::L2 # Forward LSTM 2
    d::D # Classifier
    UNK_Word_idx::Integer
end

In [ ]:
function tokenize_and_index(model, sentence::String)
    convert_to_oh(word) = Flux.unsqueeze(Flux.onehot(get(model.vocab_dict, word, get(model.vocab_dict,
                                        lowercase(word), model.UNK_Word_idx)), 1:length(model.vocab_dict)), 1)
    tokens = WordTokenizers.nltk_word_tokenize(sentence)
    ohs = convert_to_oh.(tokens)
    return ohs, tokens
end

In [ ]:
function (m::SentimentModel)(token_onehot)
    Flux.reset!(m.lstm1)
    Flux.reset!(m.lstm2)
    embed(x) = m.W_Embed * x'
    e = embed.(token_onehot)
    w = @. m.lstm2(m.lstm1(e))
    logits = m.d(m.lstm2.state[1]) # We just need the last hidden state of the LSTM
    return Flux.softmax(logits), logits
end

In [ ]:
BSON.@load "sentiment_weights.bson" trained_model

In [ ]:
sentences = ["This package is maintained by John Doe.",
             "This package is nicely maintained by John Doe.",
             "This package is poorly maintained by John Doe.",
            ]

In [ ]:
for sent in sentences
    println()
    ohs, tokens = tokenize_and_index(trained_model, sent)
    println(sent, " => ", tokens)
    scores, logits = trained_model(ohs)
    if score
    println("Positive Sentiment score: ", scores, logits)
end


In [ ]:
function predict_sentiment(sent)
    ohs, tokens = tokenize_and_index(trained_model, sent)
    scores, logits = trained_model(ohs)
    if scores[1] > 0.5
        println("\"", sent, "\" has 'positive' sentiment with score: ", scores[1])
    else
        println("\"", sent, "\" has 'negative' sentiment with score: ", scores[2])
    end
end

In [ ]:
predict_sentiment(sentences[3])